In [1]:
import konlpy
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)

In [3]:
df_train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
X_train = df_train['document'].values
y_train = df_train['label'].values

In [5]:
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)

X_test = df_test['document'].values
y_test = df_test['label'].values

In [6]:
print(len(X_train), np.bincount(y_train))

150000 [75173 74827]


In [7]:
print(len(X_test), np.bincount(y_test))

50000 [24827 25173]


In [8]:
from konlpy.tag import Okt

okt = Okt()
print(X_train[4])
print(okt.morphs(X_train[4]))

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']


In [9]:
def okt_tokenizer(text):
    return okt.morphs(text)

In [10]:
import os
from scipy.sparse import save_npz, load_npz
from sklearn.feature_extraction.text import TfidfVectorizer

if not os.path.isfile('okt_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2), 
                            min_df=3,
                            max_df=0.9,
                            tokenizer=okt_tokenizer, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_okt = tfidf.transform(X_train)
    X_test_okt = tfidf.transform(X_test)
    save_npz('okt_train.npz', X_train_okt)
    save_npz('okt_test.npz', X_test_okt)
else:
    X_train_okt = load_npz('okt_train.npz')
    X_test_okt = load_npz('okt_test.npz')

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.utils.fixes import loguniform

sgd = SGDClassifier(loss='log', random_state=1)
param_dist = {'alpha': loguniform(0.0001, 100.0)}

rsv_okt = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1)
rsv_okt.fit(X_train_okt, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:  1.1min finished


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f313ea48908>},
                   random_state=1, verbose=1)

In [12]:
print(rsv_okt.best_score_)
print(rsv_okt.best_params_)

0.8251533333333334
{'alpha': 0.0001001581395585897}


In [13]:
rsv_okt.score(X_test_okt, y_test)

0.8189

In [14]:
from soynlp.tokenizer import LTokenizer

In [15]:
lto = LTokenizer()

print(lto.tokenize(X_train[4]))

['사이몬페그의', '익살스런', '연기가', '돋보였던', '영화!스파이더맨에서', '늙어보이기만', '했던', '커스틴', '던스트가', '너무나도', '이뻐보였다']


In [16]:
from soynlp.word import WordExtractor

In [17]:
word_ext = WordExtractor()
word_ext.train(X_train)
scores = word_ext.word_scores()

training was done. used memory 1.737 Gb
all cohesion probabilities was computed. # words = 85683
all branching entropies was computed # words = 101540
all accessor variety was computed # words = 101540


In [18]:
import math

score_dict = {key: scores[key].cohesion_forward*math.exp(scores[key].right_branching_entropy) for key in scores}

In [19]:
lto = LTokenizer(scores=score_dict)

In [20]:
print(lto.tokenize(X_train[4]))

['사이', '몬페그의', '익살스', '런', '연기', '가', '돋보', '였던', '영화', '!스파이더맨에서', '늙어', '보이기만', '했던', '커스틴', '던스트가', '너무', '나도', '이뻐', '보였다']


In [21]:
def soy_tokenizer(text):
    return lto.tokenize(text)

In [22]:
if not os.path.isfile('soy_train.npz'):
    tfidf = TfidfVectorizer(ngram_range=(1, 2),
                            min_df=3,
                            max_df=0.9,
                            tokenizer=soy_tokenizer, 
                            token_pattern=None)
    tfidf.fit(X_train)
    X_train_soy = tfidf.transform(X_train)
    X_test_soy = tfidf.transform(X_test)
    save_npz('soy_train.npz', X_train_soy)
    save_npz('soy_test.npz', X_test_soy)
else:
    X_train_soy = load_npz('soy_train.npz')
    X_test_soy = load_npz('soy_test.npz')

In [23]:
rsv_soy = RandomizedSearchCV(estimator=sgd,
                             param_distributions=param_dist,
                             n_iter=50,
                             random_state=1,
                             verbose=1)
rsv_soy.fit(X_train_soy, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 250 out of 250 | elapsed:   54.6s finished


RandomizedSearchCV(estimator=SGDClassifier(loss='log', random_state=1),
                   n_iter=50,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f313ea48908>},
                   random_state=1, verbose=1)

In [24]:
print(rsv_soy.best_score_)
print(rsv_soy.best_params_)

0.8141066666666665
{'alpha': 0.0001001581395585897}


In [25]:
rsv_soy.score(X_test_soy, y_test)

0.8085